# Wrangle data

Here, I collect all the json documents downloaded from quandl API and build up one single dataset. It will be the reference dataset for the project.

In [2]:
import os
import json
import glob
from datetime import datetime as dt
import numpy as np
import pandas as pd

# data folder
data_folder = os.path.join(os.getcwd(),"..\\data")
costituents_fn = "constituents_csv.csv"
forbidden_fn = "forbidden.json"
data_folder

'C:\\dev\\dsnd-spi\\nbooks\\..\\data'

In [3]:
df_cost = pd.read_csv(os.path.join(data_folder,costituents_fn))
df_cost.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [4]:
with open(os.path.join(data_folder,forbidden_fn)) as f:
    forbidden = json.load(f)
len(forbidden)

474

In [5]:
fns     = glob.glob(os.path.join(data_folder,"*.json"))
test_fn = glob.glob(os.path.join(data_folder,"*.json"))[0]
test_fn

'C:\\dev\\dsnd-spi\\nbooks\\..\\data\\AAPL.json'

In [6]:
def load_df(fn):
    """ Load a json file into a df, and assign a ticker
    """
    ticker = fn.split('\\')[-1].split('.')[0]
    with open(fn) as f:
        test_j = json.load(f)
    cols = test_j['dataset']['column_names']
    data = np.array(test_j['dataset']['data']).T
    df = pd.DataFrame({c:d for c,d in zip(cols,data)}, dtype=float)
    df['Date'] = df['Date'].apply(lambda x:dt.strptime(x, '%Y-%m-%d'))
    df['Ticker']=ticker
    return df

load_df(test_fn)

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,Ticker
0,2017-12-28,171.00,171.850,170.480,171.080,16480187.0,0.0,1.0,165.971205,166.796208,165.466497,166.048853,16480187.0,AAPL
1,2017-12-27,170.10,170.780,169.710,170.600,21498213.0,0.0,1.0,165.097672,165.757675,164.719142,165.582968,21498213.0,AAPL
2,2017-12-26,170.80,171.470,169.679,170.570,33185536.0,0.0,1.0,165.777087,166.427383,164.689053,165.553851,33185536.0,AAPL
3,2017-12-22,174.68,175.424,174.500,175.010,16349444.0,0.0,1.0,169.542983,170.265103,169.368277,169.863278,16349444.0,AAPL
4,2017-12-21,174.17,176.020,174.100,175.010,20949896.0,0.0,1.0,169.047981,170.843576,168.980040,169.863278,20949896.0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,2013-09-09,505.00,507.920,503.480,506.170,12167400.0,0.0,1.0,64.539872,64.913053,64.345613,64.689400,85171800.0,AAPL
1086,2013-09-06,498.44,499.380,489.950,498.220,12840200.0,0.0,1.0,63.701493,63.821626,62.616456,63.673376,89881400.0,AAPL
1087,2013-09-05,500.25,500.680,493.640,495.270,8441700.0,0.0,1.0,63.932814,63.987769,63.088044,63.296361,59091900.0,AAPL
1088,2013-09-04,499.56,502.240,496.280,498.691,12322600.0,0.0,1.0,63.844631,64.187139,63.425441,63.733571,86258200.0,AAPL


In [7]:
df = pd.DataFrame()
for fn in fns:
    try:
        df = pd.concat([df, load_df(fn)])
    except:
        pass

In [8]:
df = df.reset_index(drop=True)
df

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,Ticker
0,2017-12-28,171.00,171.850,170.480,171.08,16480187.0,0.0,1.0,165.971205,166.796208,165.466497,166.048853,16480187.0,AAPL
1,2017-12-27,170.10,170.780,169.710,170.60,21498213.0,0.0,1.0,165.097672,165.757675,164.719142,165.582968,21498213.0,AAPL
2,2017-12-26,170.80,171.470,169.679,170.57,33185536.0,0.0,1.0,165.777087,166.427383,164.689053,165.553851,33185536.0,AAPL
3,2017-12-22,174.68,175.424,174.500,175.01,16349444.0,0.0,1.0,169.542983,170.265103,169.368277,169.863278,16349444.0,AAPL
4,2017-12-21,174.17,176.020,174.100,175.01,20949896.0,0.0,1.0,169.047981,170.843576,168.980040,169.863278,20949896.0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31605,2013-09-09,87.34,88.195,87.310,88.04,9297000.0,0.0,1.0,69.492212,70.172494,69.468343,70.049168,9297000.0,XOM
31606,2013-09-06,87.49,87.730,86.800,87.25,12950600.0,0.0,1.0,69.611560,69.802516,69.062560,69.420603,12950600.0,XOM
31607,2013-09-05,88.06,88.300,87.330,87.35,11740100.0,0.0,1.0,70.065081,70.256038,69.484256,69.500169,11740100.0,XOM
31608,2013-09-04,87.01,87.980,86.900,87.76,9822300.0,0.0,1.0,69.229647,70.001429,69.142125,69.826386,9822300.0,XOM


In [83]:
df.to_csv(os.path.join(data_folder,"dataset.csv"))

In [11]:
with open((os.path.join(data_folder,"tickers.json")),'w') as f:
    json.dump(list(df['Ticker'].unique()),f)